In [1]:
import os
import glob
import pandas as pd
import tensorflow as tf
import numpy as np

from IPython.display import display, HTML

In [2]:
pnumber_of_crossvalidation_run=7
cwd=os.getcwd()
visual_m_path = os.path.join(cwd,'data/cover-images_mean_feature_vectors.pkl')
textual_m_path = os.path.join(cwd,'data/text_extracted_features.pkl')
models_folder_name = os.path.join(cwd,'models','artificial_net_textual_modality')
path_to_save_test_results=os.path.join(models_folder_name, 'test_results.pkl')
model_checkpoint_path = os.path.join(cwd,models_folder_name,'gmu.ckpt')
models_folder_name_test = os.path.join(cwd,os.pardir,'text_feature_extraction','models','test',str(number_of_crossvalidation_run))
path_to_preprocessed_texts_test=os.path.join(models_folder_name_test,'book-cover_test_dataset.pkl')
summaries_folder_name=os.path.join(cwd,'summaries','artificial_net_textual_modality')


df_visual_m = pd.read_pickle(visual_m_path)
df_textual_m = pd.read_pickle(textual_m_path)

number_of_recipes=len(df_visual_m)
unique_labels=sorted(set(df_visual_m.mean_vector_labels.values))
number_of_classes=len(unique_labels)
possible_class_indices=list(range(0,number_of_classes))
labels2class_indices=dict(zip(unique_labels,possible_class_indices))
print(labels2class_indices)

Test preparation" : "0" "Calendar":"1" "Comics, graphics & novels" :"2" "Romance" :"3""Computer" "&" "technology ":"4"")
"Cookbooks, food & wine":5"Children books" :6"Science fiction and fantasy":7
"Mystery, thriller & suspense ":8"Travel ":9"Law" :10"Reference" :11
"Parenting and relationship ":12"Engineering & transportation ":13"Medical books ":14
"Crafts ,hobbies & home ":15"Religion and spirituality ":16"Sports and outdoors ":18"Self  help ":19
"Science & math ":20"Biography & memoirs" :21"Business & money ":22
"Health fitness & dieting ":23"History ":24"Teen & young Adults ":2 "Arts & photography":26"Literature & Fiction":27
"Humor & Entertainment ":28"Christian books & Bible ":29 "Political & social sciences":30


In [1]:
df_textual_m_test

Mean value: visual features  0.5558735478366387
Max value in visual feature vectors after subraction of mean value:  22.56199106092801
Mean value: textual features  0.013499930763029529
Max value in textual feature vectors after subraction of mean value:  3.519134565925813
doc_embeddings	labels	text_names
0	[-0.11329869829923188, -0.6441225998849662, -0...	Test preparation	1.txt
1	[-0.07474968697600654, -0.28306451085312634, 0...	Test preparation	10.txt
2	[-0.10599670561622081, -0.5485997736729018, 0....	Test preparation	11.txt
3	[-0.5233624146389693, -0.32503846481726917, 0....	Test preparation	12.txt
4	[-0.07889076414967888, -0.2116037304174958, 0....	Test preparation	13.txt
5	[-0.07946860561194324, -0.2566648633469014, -0...	Test preparation	14.txt
6	[-0.1800028915393702, -0.6000170525947526, 0.1...	Test preparation	15.txt
7	[-0.28211158437034184, -0.503643364081505, 0.2...	Test preparation	2.txt
8	[-0.41500734646954496, -0.40415873356834714, 0...	Test preparation	3.txt
9	[-0.386480

In [4]:
df_preprocessed_texts_test = pd.read_pickle(path_to_preprocessed_texts_test)
test_samples=df_preprocessed_texts_test.index.values.tolist()

all_samples=set(range(0,number_of_recipes))
train_samples=list(all_samples.difference(test_samples))
print("train samples: ", train_samples)
print("test samples: ", test_samples)

train samples:  [0, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 22, 24, 25, 27, 28, 29, 30, 31, 32, 33, 34, 35, 37, 38, 39, 40, 41, 42, 44, 45, 46, 47, 49, 50, 51, 52, 53, 54, 55, 56, 58, 59, 61, 62, 63, 64, 66, 67, 68, 69, 70, 71, 72, 73, 74, 76, 77, 78, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 107, 108, 109, 110, 111, 112, 113, 115, 116, 117, 118, 119, 121, 122, 123, 124, 125, 126, 127, 130, 132, 133, 134, 135, 136, 137, 138, 139, 141, 142, 143, 144, 145, 146, 147, 148, 150, 151, 152, 153, 154, 155, 156, 157, 158, 160, 161, 162, 164, 165, 166, 167, 168, 170, 171, 174, 175, 176, 177, 178, 179, 180, 181, 183, 184, 185, 186, 187, 188, 189, 190, 191, 193, 194, 195, 196, 197, 199, 200, 201, 202, 203, 204, 206, 207, 208, 209, 210, 212, 213, 214, 215, 216, 217, 218, 219, 221, 222, 223, 224, 225, 226, 227, 229, 230, 231, 232, 233, 234, 235, 237, 239, 240, 241, 242, 244, 245, 247, 248, 249, 250, 251, 252, 253, 254, 

In [5]:
df_visual_m_train=df_visual_m.iloc[train_samples]
df_visual_m_test=df_visual_m.iloc[test_samples]
df_textual_m_train=df_textual_m.iloc[train_samples]
df_textual_m_test=df_textual_m.iloc[test_samples]

visual_m_train_inputs=list(df_visual_m_train.mean_feature_vectors.values)
visual_m_test_inputs=list(df_visual_m_test.mean_feature_vectors.values)

textual_m_train_inputs=list(df_textual_m_train.doc_embeddings.values)
textual_m_test_inputs=list(df_textual_m_test.doc_embeddings.values)

train_correct_class_ids=[labels2class_indices[l] for l in df_visual_m_train.mean_vector_labels]
test_correct_class_ids=[labels2class_indices[l] for l in df_visual_m_test.mean_vector_labels]

number_of_training_samples=len(visual_m_train_inputs)
number_of_test_samples=len(visual_m_test_inputs)
len_of_visual_features_vec=len(visual_m_train_inputs[0])
len_of_textual_features_vec=len(textual_m_train_inputs[0])

print(test_correct_class_ids)
print(np.shape(visual_m_train_inputs))
print(np.shape(textual_m_train_inputs))

[0, 0, 0, 1, 1, 2, 3, 3, 3, 4, 4, 5, 6, 7, 8, 8, 9, 9, 10, 10, 11, 11, 12, 13, 13, 14, 14, 15, 15, 15, 16, 17, 17, 18, 19, 20, 21, 21, 22, 22, 22, 22, 23, 23, 23, 23, 24, 24, 25, 25, 26, 26]
(264, 202)
(264, 27)


In [6]:
batch_size=4
learning_rate=0.001
hidden_state_dim = 27
number_of_training_iterations=50000
print_valid_every=20
num_repeat_training=10

In [8]:
def remove_directory_content(path):
    files = glob.glob(path+"/*")
    for f in files:
        os.remove(f)

def create_training_batch():
    inputs_visual=[]
    inputs_textual=[]
    correct_classes=[]
    for i in range(batch_size):
        train_sample_index=np.random.choice(range(0,number_of_training_samples),1)[0]
        inputs_visual.append(visual_m_train_inputs[train_sample_index])
        inputs_textual.append(textual_m_train_inputs[train_sample_index])
        correct_classes.append(train_correct_class_ids[train_sample_index])
    return np.array(inputs_visual),np.array(inputs_textual),np.array(correct_classes)

inputs_visual,inputs_textual,correct_classes=create_training_batch()
print(np.array(correct_classes))
print(np.shape(inputs_visual))
print(np.shape(inputs_textual))
print(np.shape(correct_classes))

[12 17  0 14]
(4, 202)
(4, 27)
(4,)


In [9]:
textual = tf.placeholder(tf.float32, shape=[None,len_of_textual_features_vec], name="input_textual_modality")
target = tf.placeholder(tf.int32, shape=[None],name="input_correct_labels")

h_t = tf.layers.dense(textual, hidden_state_dim, activation=tf.nn.tanh, name="h_t")

logits = tf.layers.dense(h_t, number_of_classes, name="h")
scores = tf.nn.sigmoid(logits)

multi_class_labels=tf.one_hot(target, depth=number_of_classes)
loss = tf.reduce_mean(tf.losses.sigmoid_cross_entropy(multi_class_labels=multi_class_labels,
                                       logits=logits))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss)

correct_prediction=tf.equal(tf.argmax(logits, axis=1), tf.argmax(multi_class_labels,axis=1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

with tf.name_scope("performance"):
    loss_ph = tf.placeholder(tf.float32,shape=None,name='loss_summary')
    loss_summary = tf.summary.scalar('loss', loss_ph)
    accuracy_ph = tf.placeholder(tf.float32,shape=None,name='accuracy_summary')
    accuracy_summary = tf.summary.scalar('accuracy', accuracy_ph)

saver = tf.train.Saver()

In [10]:
best_accuracy=0

def show_validation_result(session,accuracy_res, loss_res, scores_res):
    print("Validation: Loss: ", loss_res," Accuracy: ", accuracy_res)
    global best_accuracy
    if accuracy_res > best_accuracy:
        best_accuracy=accuracy_res
        df_performance=pd.DataFrame(data={'Accuracy':[accuracy_res], 'Loss':[loss_res]})
        class_scores=list(np.around(scores_res,4))
        predicted_labels = [np.argmax(one_recipe_class_scores) for one_recipe_class_scores in class_scores]
        df_scores=pd.DataFrame(data={'Class scores':class_scores,
                               'Predicted labels':predicted_labels,
                               'Correct labels':test_correct_class_ids,
                               })
        df_res=pd.concat([df_scores, df_performance], axis=1)
        df_res.to_pickle(path_to_save_test_results)
        saver.save(session, model_checkpoint_path)
        display(df_res)
        
    

def train():
        session=tf.InteractiveSession()
        summ_writer = tf.summary.FileWriter(summaries_folder_name, session.graph)
        session.run(tf.global_variables_initializer())
    
        print("Start model training")
    
        for train_iter in range(number_of_training_iterations):           
            inputs_visual,inputs_textual,correct_classes=create_training_batch()
            _, l = session.run([train_op, loss], {textual: inputs_textual,target: correct_classes})
            print(train_iter, ": Training: loss: ", l)
            summ_loss = session.run(loss_summary, feed_dict={loss_ph:l})
            summ_writer.add_summary(summ_loss, train_iter)
        
            if (train_iter)%print_valid_every==0:                        
                accuracy_res, loss_res, scores_res = session.run([accuracy, loss, scores],
                                                                          {textual: textual_m_test_inputs,
                                                                           target: test_correct_class_ids})        
                
                show_validation_result(session,accuracy_res, loss_res, scores_res)
                summ_accuracy = session.run(accuracy_summary, feed_dict={accuracy_ph:accuracy_res})
                summ_writer.add_summary(summ_accuracy, train_iter)
                
                if accuracy_res==1.0:
                    return 0
                
        session.close()
        return -1

In [11]:
remove_directory_content(summaries_folder_name)
train()

Start model training
0 : Training: loss:  0.6739862
Validation: Loss:  0.7004497  Accuracy:  0.09615385


,Class scores,Correct labels,Predicted labels,Accuracy,Loss
0,"[0.5773, 0.4286, 0.5809, 0.3986, 0.3182, 0.547...",0,2,0.096154,0.70045
1,"[0.5475, 0.3959, 0.5859, 0.4025, 0.3352, 0.440...",0,26,NaN,NaN
2,"[0.5055, 0.568, 0.6421, 0.4399, 0.509, 0.5851,...",0,8,NaN,NaN
3,"[0.5237, 0.4336, 0.5391, 0.5193, 0.4283, 0.460...",1,25,NaN,NaN
4,"[0.4734, 0.4583, 0.5588, 0.5682, 0.4996, 0.393...",1,10,NaN,NaN
5,"[0.4462, 0.5193, 0.4865, 0.5024, 0.5403, 0.453...",2,10,NaN,NaN
6,"[0.5835, 0.4735, 0.5664, 0.4125, 0.5393, 0.502...",3,9,NaN,NaN
7,"[0.6118, 0.5023, 0.5633, 0.4653, 0.5154, 0.610...",3,16,NaN,NaN
8,"[0.5649, 0.5391, 0.6213, 0.4288, 0.5845, 0.666...",3,22,NaN,NaN
9,"[0.3802, 0.5371, 0.5429, 0.5477, 0.5922, 0.330...",4,12,NaN,NaN


1 : Training: loss:  0.67612404
2 : Training: loss:  0.6831408
3 : Training: loss:  0.6744434
4 : Training: loss:  0.69870806
5 : Training: loss:  0.71235013
6 : Training: loss:  0.6852699
7 : Training: loss:  0.6756886
8 : Training: loss:  0.728265
9 : Training: loss:  0.67974
10 : Training: loss:  0.6878726
11 : Training: loss:  0.71393424
12 : Training: loss:  0.68292415
13 : Training: loss:  0.7075807
14 : Training: loss:  0.71384877
15 : Training: loss:  0.6776031
16 : Training: loss:  0.6996258
17 : Training: loss:  0.68505996
18 : Training: loss:  0.6895129
19 : Training: loss:  0.7150982
20 : Training: loss:  0.67453295
Validation: Loss:  0.6802102  Accuracy:  0.13461539


,Class scores,Correct labels,Predicted labels,Accuracy,Loss
0,"[0.5783, 0.4163, 0.5753, 0.3916, 0.311, 0.547,...",0,0,0.134615,0.68021
1,"[0.549, 0.385, 0.5785, 0.3964, 0.3314, 0.4391,...",0,26,NaN,NaN
2,"[0.511, 0.5652, 0.6367, 0.4415, 0.5075, 0.5916...",0,8,NaN,NaN
3,"[0.5201, 0.4146, 0.5294, 0.496, 0.4138, 0.4577...",1,25,NaN,NaN
4,"[0.4638, 0.438, 0.5472, 0.5425, 0.4808, 0.3867...",1,10,NaN,NaN
5,"[0.4376, 0.4955, 0.4834, 0.4798, 0.5232, 0.441...",2,10,NaN,NaN
6,"[0.5801, 0.4609, 0.5587, 0.4087, 0.5343, 0.498...",3,16,NaN,NaN
7,"[0.6042, 0.4906, 0.5608, 0.4598, 0.5027, 0.602...",3,16,NaN,NaN
8,"[0.5649, 0.54, 0.623, 0.4373, 0.583, 0.6655, 0...",3,22,NaN,NaN
9,"[0.3812, 0.5121, 0.5274, 0.5275, 0.5791, 0.329...",4,12,NaN,NaN


21 : Training: loss:  0.70796126
22 : Training: loss:  0.68469197
23 : Training: loss:  0.6830398
24 : Training: loss:  0.6757053
25 : Training: loss:  0.6668065
26 : Training: loss:  0.66289115
27 : Training: loss:  0.67156476
28 : Training: loss:  0.6964206
29 : Training: loss:  0.6584073
30 : Training: loss:  0.68664366
31 : Training: loss:  0.65477854
32 : Training: loss:  0.64978904
33 : Training: loss:  0.645622
34 : Training: loss:  0.68069196
35 : Training: loss:  0.6468587
36 : Training: loss:  0.6681946
37 : Training: loss:  0.650808
38 : Training: loss:  0.66384685
39 : Training: loss:  0.6597024
40 : Training: loss:  0.67060024
Validation: Loss:  0.6598941  Accuracy:  0.13461539
41 : Training: loss:  0.6422047
42 : Training: loss:  0.6511383
43 : Training: loss:  0.67141837
44 : Training: loss:  0.6550969
45 : Training: loss:  0.6611154
46 : Training: loss:  0.65435475
47 : Training: loss:  0.6422294
48 : Training: loss:  0.67487186
49 : Training: loss:  0.6582981
50 : Trai

...

4282 : Training: loss:  0.040006973
4283 : Training: loss:  0.022238828
4284 : Training: loss:  0.03637572
4285 : Training: loss:  0.015797248
4286 : Training: loss:  0.019449027
4287 : Training: loss:  0.016601725
4288 : Training: loss:  0.018393446
4289 : Training: loss:  0.019942882
4290 : Training: loss:  0.0135175055
4291 : Training: loss:  0.02000258
4292 : Training: loss:  0.032054376
4293 : Training: loss:  0.029999088
4294 : Training: loss:  0.0241841
4295 : Training: loss:  0.028037168
4296 : Training: loss:  0.0155497175
4297 : Training: loss:  0.03153431
4298 : Training: loss:  0.015687997
4299 : Training: loss:  0.01937735
4300 : Training: loss:  0.012476804
Validation: Loss:  0.035707578  Accuracy:  0.9230769
4301 : Training: loss:  0.012492628
4302 : Training: loss:  0.016505416
4303 : Training: loss:  0.023118315
4304 : Training: loss:  0.01738112
4305 : Training: loss:  0.01699498
4306 : Training: loss:  0.027505903
4307 : Training: loss:  0.016474368
4308 : Training: 

4514 : Training: loss:  0.017291984
4515 : Training: loss:  0.015394275
4516 : Training: loss:  0.028609581
4517 : Training: loss:  0.01632942
4518 : Training: loss:  0.015233531
4519 : Training: loss:  0.017164495
4520 : Training: loss:  0.011963902
Validation: Loss:  0.034182508  Accuracy:  0.9423077
4521 : Training: loss:  0.01983238
4522 : Training: loss:  0.014615302
4523 : Training: loss:  0.012154963
4524 : Training: loss:  0.033844642
4525 : Training: loss:  0.01489918
4526 : Training: loss:  0.017359864
4527 : Training: loss:  0.017718403
4528 : Training: loss:  0.016848795
4529 : Training: loss:  0.012242258
4530 : Training: loss:  0.019297384
4531 : Training: loss:  0.020906445
4532 : Training: loss:  0.016550602
4533 : Training: loss:  0.020196632
4534 : Training: loss:  0.024461217
4535 : Training: loss:  0.021924876
4536 : Training: loss:  0.021379974
4537 : Training: loss:  0.022440676
4538 : Training: loss:  0.009550242
4539 : Training: loss:  0.024084358
4540 : Trainin

4729 : Training: loss:  0.02319815
4730 : Training: loss:  0.011734706
4731 : Training: loss:  0.01044295
4732 : Training: loss:  0.023882238
4733 : Training: loss:  0.011827918
4734 : Training: loss:  0.025699696
4735 : Training: loss:  0.014816135
4736 : Training: loss:  0.020043803
4737 : Training: loss:  0.0141677335
4738 : Training: loss:  0.013350153
4739 : Training: loss:  0.023755394
4740 : Training: loss:  0.014031025
Validation: Loss:  0.033128697  Accuracy:  0.90384614
4741 : Training: loss:  0.011269114
4742 : Training: loss:  0.018458232
4743 : Training: loss:  0.01700378
4744 : Training: loss:  0.02311618
4745 : Training: loss:  0.017157929
4746 : Training: loss:  0.0144562
4747 : Training: loss:  0.022384275
4748 : Training: loss:  0.014732595
4749 : Training: loss:  0.013269359
4750 : Training: loss:  0.01694579
4751 : Training: loss:  0.019831002
4752 : Training: loss:  0.017509434
4753 : Training: loss:  0.0114271045
4754 : Training: loss:  0.012258769
4755 : Training

4959 : Training: loss:  0.014553553
4960 : Training: loss:  0.014014456
Validation: Loss:  0.031651754  Accuracy:  0.9423077
4961 : Training: loss:  0.0202997
4962 : Training: loss:  0.010047115
4963 : Training: loss:  0.013555407
4964 : Training: loss:  0.010543639
4965 : Training: loss:  0.009358325
4966 : Training: loss:  0.017693108
4967 : Training: loss:  0.020235958
4968 : Training: loss:  0.019750537
4969 : Training: loss:  0.017574323
4970 : Training: loss:  0.016858624
4971 : Training: loss:  0.022464732
4972 : Training: loss:  0.01596108
4973 : Training: loss:  0.018337846
4974 : Training: loss:  0.009897653
4975 : Training: loss:  0.00795775
4976 : Training: loss:  0.012529233
4977 : Training: loss:  0.020157853


KeyboardInterrupt: 